# Generate simplified documentation pages
Use data available in Houdini documentation.

In [1]:
import sys
import pathlib
import zipfile
import json
import html
import htmltag

from hfs import HFS
sys.path.append(str(HFS / 'houdini' / 'python3.11libs'))
sys.path.append(str(HFS / 'python311' / 'lib' / 'site-packages'))
from bookish.wiki.wikipages import parse_to_root as parse_wikipage

In [2]:
# Open expressions.zip containing various helpcards including functions.
# Parse Wiki files into JSON using Bookish parser.
expressions = {}

with zipfile.ZipFile(HFS / 'houdini' / 'help' / 'expressions.zip') as z:
    for path in z.namelist():
        path = pathlib.PurePath(path)
        with z.open(path.as_posix()) as f:
            name = path.stem
            markup = f.read().decode()
            expressions[name] = parse_wikipage(markup)

expressions['index'] = expressions['index_']

len(expressions)

475

In [3]:
# Inspect Bookish types.
def types(node):
    '''Recursively extract all 'type' values from Bookish tree.'''
    if type(node) is dict:
        try:
            yield node['type']
        except:
            print(node)

        for k in node:
            if type(node[k]) is list:
                for i in node[k]:
                    for j in types(i):
                        yield j

def all_types(expressions):
    for f in expressions:
        for t in types(expressions[f]):
            yield t


bookish_types = set(all_types(expressions))
print(' '.join(sorted(bookish_types)))
print('Total:', len(bookish_types))

bullet code dt em examples_section h include item keys link note para pre prop q related_section root strong summary tip title ui usage var
Total: 24


In [4]:
class Wrapper:
    '''Convert Bookish JSON structure to html.'''

    def __init__(self):
        # Trivial type handlers.
        from htmltag import body, code, div, em, h1, h2, p, span, strong

        self.root = lambda n: body(self(n['body']))

        self.arg = lambda n: div(code(self(n['text'])), _class='argument')
        self.bullet = lambda n: div(self(n['text']), _class='related')
        self.cell = lambda n: span(self(n['text']))
        self.code = lambda n: code(self(n['text']))
        self.em = lambda n: em(self(n['text']))
        self.strong = lambda n: strong(self(n['text']))
        self.summary = lambda n: p(self(n['text']), _class='summary')
        self.usage = lambda n: div(self(n['text']), _class='usage')
        self.var = lambda n: code(self(n['text']), _class='var')
        self.xml = lambda n: self(n['text'])
        self.q = lambda n: self(n['text'])

        self.h = lambda n: h2(self(n['text']))
        self.note = lambda n: h2('Note')
        self.tip = lambda n: h2('Tip')
        self.warning = lambda n: h2('Warning')
        self.returns = lambda n: h2('Returns')

        self.examples_section = lambda n: h2('Examples')
        self.related_section = lambda n: h2('Related')
        self.subtopics_section = lambda n: ''

        self.box = lambda n: ''
        self.keys = lambda n: ''
        self.list = lambda n: ''
        self.null = lambda n: ''
        self.ord = lambda n: ''
        self.pxml = lambda n: ''
        self.supertitle = lambda n: ''
        self.include = lambda n: ''

        self.dt = self.arg
        self.item = self.bullet
        self.ui = self.strong
        self.varg = self.arg
        self.returnss = self.returns  # Patch someone's typo.
        
    def para(self, n):
        from htmltag import p
        if 'text' in n:
            return p(self(n['text']))
        return ''
    
    def title(self, n):
        from htmltag import a, h1
        name = self(n['text'])
        url = f'https://www.sidefx.com/docs/houdini/expressions/{name}.html'
        return h1(a(name, href=url))

    def pre(self, n):
        from htmltag import HTML, code, div
        text = self(n['text']).strip()
        lines = text.split('\n')
        tags = [code(HTML(l), _class='codeline') for l in lines]
        return div(tags, _class='codeblock')

    def link(self, n):
        from htmltag import a

        if n['scheme'] == 'Include':
            key = n['value']
            if key == '_index_argument':
                return self(expressions[key]['body'] )

            print('Unknown include:', n)
            return ''

        base = {
            'Exp': 'https://www.sidefx.com/docs/houdini/expressions/',
            'Hom': 'https://www.sidefx.com/docs/houdini/hom/hou/',
            'Hprop': 'https://www.sidefx.com/docs/houdini/props/mantra#',
            'Image': 'https://www.sidefx.com/docs/houdini',
            'Node': 'https://www.sidefx.com/docs/houdini/nodes/',
            'Vex': 'https://www.sidefx.com/docs/houdini/vex/functions/',
            'Cmd': 'https://www.sidefx.com/docs/houdini/commands/',
            'Wp': 'https://en.wikipedia.org/wiki/',
            None: 'https://www.sidefx.com/docs/houdini',
        }[n['scheme']]

        rest = n['value']
        if n['scheme'] == 'Hom':
            rest = rest.rsplit('.')[-1]

        content = n['value'] if not n['text'] else n['text']
        return a(self(content), href=base+rest)

    def prop(self, n):
        from htmltag import div, span
      
        # Currently used only to list deprecated functions.
        if n['name'] in ('index', 'status'):
            return ''
        
        # Version info. Could hint old users to give some attention to the function.
        elif n['name'] == 'since':
            t = span(n['value'], _class='pillow')
            t = span(t, _class='padder')
            return t

        # Common unuseful props.
        if n['name'] in ('type', 'group', 'id', 'redirect', 'minitoc', 'category'):
            return ''

        print('Unknown prop:', n)
        return ''

    def __call__(self, node):
        if isinstance(node, dict):
            if 'type' in node:  # Workaround bookish Empty node.
                return getattr(self, node['type'])(node)
            else:
                return ''
        elif isinstance(node, list):
            return htmltag.HTML(''.join(self(i) for i in node))
        else:
            return html.escape(node, quote=False)


wrapper = Wrapper()


# Debug.
# print(yaml.dump(expressions['pointdist']))
# markup = wrapper(expressions['pointdist'])
# soup = bs4.BeautifulSoup(markup, 'html.parser')
# print(soup.prettify())


# Process some functions.
# helpcards = {
#     'ch': wrapper(expressions['ch']),
#     'pointdist': wrapper(expressions['pointdist']),
#     'property': wrapper(expressions['property']),
# }


# Process all functions.
helpcards = {}
for e in expressions:
    if e == '_index_argument':
        continue
    helpcards[e] = wrapper(expressions[e])


# Dump on disk.
with open('helpcards.json', 'w') as f:
    json.dump(helpcards, f, indent=4)